In [156]:
import pandas as pd
import category_encoders as ce
import csv
from xgboost import XGBClassifier
import numpy as np
from sklearn import *
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
import category_encoders as ce
import xgboost
import collections
pd.options.display.max_rows = 999
from sklearn import preprocessing

Importing some data

In [158]:
path="D:/downloads/scoremouna.csv"

In [194]:
train=pd.read_csv(path,sep=';')
train["id"]=np.array(range(train.shape[0]))

In [189]:
train.columns

Index(['situation', 'nb_enfants', 'civilite', 'DN', 'STAT_PRO', 'Budget',
       'taste', 'duration_in_days', 'target'],
      dtype='object')

In [163]:
missing_columns=list(train.columns)
missing_columns.remove("date_depot")
missing_columns.remove("campagne_id")
missing_columns.remove("id_organisme")
missing_columns.remove("prospects_id")
missing_columns.remove("SIGN")
missing_columns.remove("RESIL")


In [164]:
missing_columns

['commerciaux_id',
 'DN',
 'situation',
 'nb_enfants',
 'civilite',
 'code_dept',
 'ville',
 'longitude',
 'latitude',
 'STAT_PRO',
 'ANNEE_NES_PROS',
 'PROFESSION_PROS',
 'RO_PROS',
 'RO_CONJOINT',
 'ANNEE_NAIS_CONJOINT',
 'cmp',
 'mca',
 'nb_adh']

In [165]:
for i in missing_columns:
    train[i].fillna(train.loc[1,i],inplace=True)

In [186]:
train["STAT_PRO"].fillna(train.loc[15,"STAT_PRO"],inplace=True)


In [167]:
train=train[['situation','nb_enfants','civilite','DN','STAT_PRO']].copy()

Filling the dataframe with some synthetic data

In [168]:
Budget=np.random.randint(low=0,high=1001,size=train.shape[0])
taste=np.random.randint(low=0,high=2,size=train.shape[0])
duration_in_days=np.random.randint(low=1,high=61,size=train.shape[0])



In [169]:
train["Budget"]=Budget
train["taste"]=taste
train["duration_in_days"]=duration_in_days

In [170]:
train

,situation,nb_enfants,civilite,DN,STAT_PRO,Budget,taste,duration_in_days
0,N.R,0.0,M.,1987.0,Sans emploi,961,1,21
1,N.R,0.0,Mme,1963.0,Sans emploi,810,0,53
2,N.R,0.0,Mme,1947.0,Retraité(e),184,1,20
3,Celibataire,0.0,M.,1946.0,Retraité(e),769,0,40
4,N.R,0.0,Mme,1947.0,Retraité(e),904,1,52
...,...,...,...,...,...,...,...,...
80726,Marie(e),0.0,M.,1996.0,Salarié(e) du privé (-55 ans) en CDI,328,1,36
80727,N.R,0.0,M.,1929.0,Retraité(e),299,0,40
80728,N.R,0.0,Mme,1929.0,Retraité(e),823,0,6
80729,N.R,0.0,M.,1995.0,Salarié(e) du privé (-55 ans) en CDI,792,0,29


In [171]:
train.DN=train.DN*0.1
train.DN=train.DN.round()
train.DN=train.DN*10
train.DN=train.DN.astype(int)

In [172]:
train

,situation,nb_enfants,civilite,DN,STAT_PRO,Budget,taste,duration_in_days
0,N.R,0.0,M.,1990,Sans emploi,961,1,21
1,N.R,0.0,Mme,1960,Sans emploi,810,0,53
2,N.R,0.0,Mme,1950,Retraité(e),184,1,20
3,Celibataire,0.0,M.,1950,Retraité(e),769,0,40
4,N.R,0.0,Mme,1950,Retraité(e),904,1,52
...,...,...,...,...,...,...,...,...
80726,Marie(e),0.0,M.,2000,Salarié(e) du privé (-55 ans) en CDI,328,1,36
80727,N.R,0.0,M.,1930,Retraité(e),299,0,40
80728,N.R,0.0,Mme,1930,Retraité(e),823,0,6
80729,N.R,0.0,M.,2000,Salarié(e) du privé (-55 ans) en CDI,792,0,29


In [173]:
Budget_rounded=train.Budget.values
Budget_rounded=Budget_rounded*0.01
Budget_rounded=np.where(Budget_rounded<1,1,Budget_rounded)
Budget_rounded=Budget_rounded.round()
Budget_rounded=Budget_rounded*100
train.Budget=Budget_rounded

In [174]:
train.Budget

0        1000.0
1         800.0
2         200.0
3         800.0
4         900.0
          ...  
80726     300.0
80727     300.0
80728     800.0
80729     800.0
80730     100.0
Name: Budget, Length: 80731, dtype: float64

In [175]:
Y_train=np.random.randint(low=0,high=21,size=train.shape[0])
Y_train=pd.DataFrame(data=Y_train)
train["target"]=Y_train

In [176]:
encoder=ce.OrdinalEncoder()
encoder.fit(train)
X_train=encoder.transform(train)

In [177]:
model = XGBClassifier(max_depth=7,colsample_bytree=0.5,min_child_weight=2,
                      learning_rate=0.05,objective = "multi:softprob",n_estimators=148)

Generating some predictions to demonstrate the feasibility of the idea. No tuning was done since this is a synthetic data.

In [180]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train, Y_train, test_size=0.2)
model.fit(x_train,y_train)
ypred=model.predict(x_test)


D:\Anaconda3\envs\V02 With Keras\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda3\envs\V02 With Keras\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Printing some predictions

In [198]:
x_test_indexes=x_test.index


In [201]:
print("user id {} : value of the target: {}".format(x_test_indexes[0],ypred[0]))
print("user id {} : value of the target: {}".format(x_test_indexes[1],ypred[1]))
print("user id {} : value of the target: {}".format(x_test_indexes[2],ypred[2]))
print("user id {} : value of the target: {}".format(x_test_indexes[3],ypred[3]))

user id 37239 : value of the target: 4
user id 11511 : value of the target: 12
user id 12380 : value of the target: 6
user id 69038 : value of the target: 5
